In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from gpt_index.composability.joint_qa_summary import QASummaryGraphBuilder
from gpt_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from gpt_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
reader = SimpleDirectoryReader('../paul_graham_essay/data')
documents = reader.load_data()

In [5]:
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context_gpt4 = ServiceContext.from_defaults(llm_predictor=llm_predictor_gpt4, chunk_size_limit=1024)

llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt, chunk_size_limit=1024)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [6]:
# NOTE: can also specify an existing docstore, service context, summary text, qa_text, etc.
graph_builder = QASummaryGraphBuilder(service_context=service_context_gpt4)
graph = graph_builder.build_graph_from_documents(documents)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 20729 tokens
> [build_index_from_nodes] Total embedding token usage: 20729 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage:

In [9]:
query_engine = graph.as_query_engine(
    response_mode='tree_summarize',
    use_async=True,
    verbose=True,
    service_context=service_context_gpt4
)

In [10]:

response = query_engine.query(
    "Can you give me a summary of the author's life?", 
)

INFO:gpt_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [2]/[2]
>[Level 0] Selected node: [2]/[2]
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 172 tokens
> [retrieve] Total LLM token usage: 172 tokens
INFO:gpt_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 0 tokens
> [retrieve] Total embedding token usage: 0 tokens
INFO:gpt_index.indices.common_tree.base:> Building index from nodes: 6 chunks
> Building index from nodes: 6 chunks
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4254 request_id=99860fad320ba2443dcbed71341019cb response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=4254 request_id=99860fad320ba2443dcbed71341019cb response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=15317 request_id=2d14e999f947131263d969d2c5fa63c2 

In [11]:
print(response)

The author's life has been a diverse journey through various interests and pursuits. They began with a passion for programming and writing short stories, studied artificial intelligence in college, and later explored art by attending art schools in Italy and the US. They worked as a freelance Lisp programmer and studio assistant before co-founding Viaweb, an online store builder that was sold to Yahoo. The author then started writing popular essays online and co-founded Y Combinator, a startup accelerator. Their life has spanned fields such as art, software, venture capital, and essay writing, driven by curiosity and independent-mindedness.


In [15]:
response = query_engine.query(
    "What did the author do growing up?", 
)

>[Level 0] Current response: ANSWER: 1

The question, "What did the author do growing up?" asks for specific context from documents about the author's experiences or activities during their childhood. Choice 1 mentions retrieval of specific context from documents, which is more in line with answering this type of question than summarization queries mentioned in Choice 2.
INFO:llama_index.indices.tree.leaf_query:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Node [1] Summary text: Use this index for queries that require retrieval of specific context from documents.
> Got node text: Growing up, the author worked on writing short stories and programming on the IBM 1401, a computer used in their school district. They also experimented with programming on a TRS-80 microcomputer, ...


In [16]:
print(response)

Growing up, the author worked on writing short stories and programming on the IBM 1401 computer. They also experimented with programming on a TRS-80 microcomputer, creating simple games, a model rocket prediction program, and a word processor for their father.


In [17]:
response = query_engine.query(
    "What did the author do during his time in art school?", 
)

>[Level 0] Current response: ANSWER: 1

This summary was selected in relation to the question because it involves retrieval of specific context from documents, which is necessary to know what the author did during his time in art school. Choice 2 is focused on summarization queries, which isn't related to the specific detail needed to answer the question.
INFO:llama_index.indices.tree.leaf_query:>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Selected node: [1]/[1]
>[Level 0] Node [1] Summary text: Use this index for queries that require retrieval of specific context from documents.
> Got node text: The author took classes in fundamental subjects like drawing, color, and design during the foundation program at RISD. He also prepared for the entrance exam at the Accademia di Belli Arti in Flore...


In [18]:
print(response)

During his time in art school, the author took classes in fundamental subjects, such as drawing, color, and design, as part of the foundation program at RISD. He also prepared for the entrance exam at the Accademia di Belli Arti in Florence, which involved learning Italian.
